In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

from os.path import join
import os
import sys
sys.path.append('../')
from ptype.soundings.xr_map_reduce import get_dirpaths

## Select forecast hour:

In [2]:
case_study = 'kentucky'
model = 'rap'
day = '20220223'
path = f"/glade/scratch/dkimpara/ptype_case_studies/{case_study}/{model}/{day}"
paths = get_dirpaths(model, path)

In [3]:
def get_f01(paths):
    f01s = []
    for path in paths:
        for dirpath, dirnames, filenames in os.walk(path):
            for filename in filenames:
                if 'f01' in filename:
                    f01s.append(join(dirpath,filename))
                
    return f01s

In [4]:
ds = xr.open_mfdataset(get_f01(paths),
                       concat_dim="valid_time",
                      combine="nested",
                      decode_cf=False)

In [5]:
uniform = np.ones(4) / 4
smallest_dist = np.inf
best_coord = (0,0)

for x_cord in ds.x:
    for y_cord in ds.y:
        subds = ds.sel(x=x_cord, y=x_cord)
        _, counts = np.unique(subds.values, return_counts=True)
        dist_from_unif = np.sum((uniform - counts / np.sum(counts)) ** 2)
        
        if dist_from_unif < smallest_dist:
            smallest_dist = dist_from_unif
            best_coord = (x_cord, y_cord)

IndexError: Index 337 is out of bounds for axis 3 with size 337

In [ ]:
from joblib import dump
dump(best_coord, "best_coord.dump")